### **Financial Analysis:** Assignment 2

Testing the performance of indicators

---
### **FInancial Indicators Tested:**




> On Balance VOlume (OBV)



> Relative strength index (RSI)

The indicators were tested from February 2022 to March 2022, as this is when the war begins.


In [ ]:
pip install yfinance

In [38]:
# importing necessary libraries

import yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

In [39]:
# defining a function to import the tickers data from yahoo finance

def get_data(ticker_name:str, start_date:str, end_date:str, interval: str) -> pd.DataFrame:

  """This function will returns the data for a given ticker, start date, end date and interval"""

  data = yf.download(tickers=ticker_name, start = start_date , end = end_date, interval=interval,prepost= True)

  return data

### **On Balance Volume (OBV)**

In [54]:
# defining a function that calculates and plots the Buy and sell signals wrt OBV

def test_obv(ticker_name:str, start_date:str, end_date:str, interval: str):

  # using the get data function to download data

  data = get_data(ticker_name, start_date, end_date, interval)

  # Calculating the On-Balance Volume (OBV) indicator
  close_prices = data['Adj Close']
  volume = data['Volume']
  change_in_price = close_prices.diff()
  direction = pd.Series([1 if x >= 0 else -1 for x in change_in_price], index=change_in_price.index)
  direction[0] = 0
  obv = (direction * volume).cumsum()

  # Define the buy/sell thresholds
  buy_threshold = 0
  sell_threshold = 0

  # Determine the buy/sell signals
  obv_diff = obv.diff()
  buy_signals = (obv > buy_threshold) & (obv_diff > 0)
  sell_signals = (obv < sell_threshold) & (obv_diff < 0)

  # Generate the chart
  fig = go.Figure()

  # Add the stock price trace
  fig.add_trace(go.Scatter(
      x=data.index,
      y=close_prices,
      name="Stock price",
      line=dict(color='skyblue')
  ))

  # adding candle stick plot
  fig.add_trace(go.Candlestick(x=data.index,
                              open=data['Open'],
                              high=data['High'],
                              low=data['Low'],
                              close=data['Close'],
                              name=f'{(ticker_name)} '))

  # Add the OBV trace
  fig.add_trace(go.Scatter(
      x=data.index,
      y=obv,
      name="OBV",
      yaxis="y2",
      line=dict(color='orange')
  ))

  # Add the buy/sell signals
  buys = close_prices[buy_signals]
  sells = close_prices[sell_signals]

  fig.add_trace(go.Scatter(
      x=buys.index,
      y=buys,
      mode="markers",
      marker=dict(symbol='triangle-up', size=10, color="green"),
      name="Buy"
  ))

  fig.add_trace(go.Scatter(
      x=sells.index,
      y=sells,
      mode="markers",
      marker=dict(symbol='triangle-down', size=10, color="red"),
      name="Sell"
  ))

  # Add the chart layout
  fig.update_layout(
      title=f"{ticker_name} stock prices and OBV indicator ({start_date} to {end_date})",
      xaxis=dict(title="Date"),
      yaxis=dict(title="Stock price"),
      yaxis2=dict(title="OBV", overlaying="y", side="right")
  )

  fig.show()

In [55]:
# applying the function on all tickers
test_obv('FSLR',"2022-01-01", "2023-01-31", "1d")
test_obv('ENPH',"2022-01-01", "2023-01-31", "1d")
test_obv('XOM',"2022-01-01", "2023-01-31", "1d")
test_obv('TAN',"2022-01-01", "2023-01-31", "1d")
test_obv('VDE',"2022-01-01", "2023-01-31", "1d")
test_obv('FAN',"2022-01-01", "2023-01-31", "1d")
test_obv('ORSTED.CO',"2022-01-01", "2023-01-31", "1d")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


###**Relative Strength Index (RSI)**

In [61]:
# defining a function that calculates and plots the Buy and sell signals wrt RSI

def test_rsi(ticker_name:str, start_date:str, end_date:str, interval: str):

  # using the get data function to download data

  data = get_data(ticker_name, start_date, end_date, interval)

  # calculating RSI

  periods = 14

  close_delta = data['Close'].diff()
  gain = close_delta.where(close_delta > 0, 0)
  loss = -close_delta.where(close_delta < 0, 0)
  avg_gain = gain.rolling(window=periods).mean()
  avg_loss = loss.rolling(window=periods).mean()
  rs = avg_gain / avg_loss
  rsi = 100.0 - (100.0 / (1.0 + rs))

  data['RSI'] = rsi

  # Add Buy/Sell signals to the plot
  data['Buy'] = ((data['RSI'] < 30) & (data['RSI'].shift(1) >= 30))
  data['Sell'] = ((data['RSI'] > 70) & (data['RSI'].shift(1) <= 70))

  # Plot the data with Buy/Sell signals using Plotly
  fig = go.Figure()

  # adding candle stick plot
  fig.add_trace(go.Candlestick(x=data.index,
                              open=data['Open'],
                              high=data['High'],
                              low=data['Low'],
                              close=data['Close'],
                              name=f'{(ticker_name)} '))

  fig.add_trace(go.Scatter(x=data.index,
                           y=data['Close'],
                           name='Stock Price',
                           line=dict(color='skyblue')))
  
  fig.add_trace(go.Scatter(x=data.index,
                           y=data['RSI'],
                           name='RSI',
                           yaxis="y2",
                           line=dict(color='orange')))
  
  fig.add_trace(go.Scatter(x=data.loc[data['Buy']].index,
                           y=data['Close'][data['Buy']],
                           mode='markers',
                           marker=dict(color='green',
                                       symbol='triangle-up',
                                       size=10),
                                       name='Buy'))
  
  fig.add_trace(go.Scatter(x=data.loc[data['Sell']].index,
                           y=data['Close'][data['Sell']],
                           mode='markers',
                           marker=dict(color='red',
                                       symbol='triangle-down',
                                       size=10),
                                       name='Sell'))
  
  fig.update_layout(title=f'({ticker_name}) with RSI Indicator and Buy/Sell signals')
  # Adding the chart layout
  fig.update_layout(
    xaxis=dict(title="Date"),
    yaxis=dict(title="Stock price"),
    yaxis2=dict(title="RSI", overlaying="y", side="right")
  )

  fig.show()

In [ ]:
# applying the function on all tickers
test_rsi('FSLR',"2022-01-01", "2023-01-31", "1d")
test_rsi('ENPH',"2022-01-01", "2023-01-31", "1d")
test_rsi('XOM',"2022-01-01", "2023-01-31", "1d")
test_rsi('TAN',"2022-01-01", "2023-01-31", "1d")
test_rsi('VDE',"2022-01-01", "2023-01-31", "1d")
test_rsi('FAN',"2022-01-01", "2023-01-31", "1d")
test_rsi('ORSTED.CO',"2022-01-01", "2023-01-31", "1d")

### **Returns on Investment (ROI) wrt Indicators**

In [108]:
# calculating profit for both indicator for all tickers

tickers = ['FSLR', 'ENPH','TAN','FAN','ORSTED.CO']

amount_invested = 10000
start_date = "2022-02-01"
end_date = "2022-05-31"

OBV_Profit = []
RSI_Profit = []

for ticker in tickers:

  data = get_data(ticker,start_date, end_date, "1d")

  # Calculate the On-Balance Volume (OBV) indicator
  close_prices = data['Adj Close']
  volume = data['Volume']
  change_in_price = close_prices.diff()

  # Add check for zero volume
  direction = pd.Series([0 if x == 0 else 1 if x >= 0 else -1 for x in change_in_price], index=change_in_price.index)
  obv = (direction * volume).cumsum()

  data['OBV'] = obv

  # Calculate the daily change in OBV
  data['OBV_change'] = data['OBV'].diff()

  # Define the buy and sell signals based on the OBV values
  data['signal'] = 0
  data.loc[data['OBV_change'] > 0, 'signal'] = 1  # buy signal
  data.loc[data['OBV_change'] < 0, 'signal'] = -1  # sell signal

  # Calculate the profit based on the buy/sell signals
  data['profit'] = data['Close'].pct_change() * data['signal'].shift(1)
  total_profit = data['profit'].sum()

  tp = round(total_profit * amount_invested,2)

  OBV_Profit.append(tp)

  # RSI
  periods = 14

  close_delta = data['Close'].diff()
  gain = close_delta.where(close_delta > 0, 0)
  loss = -close_delta.where(close_delta < 0, 0)
  avg_gain = gain.rolling(window=periods).mean()
  avg_loss = loss.rolling(window=periods).mean()
  rs = avg_gain / avg_loss
  rsi = 100.0 - (100.0 / (1.0 + rs))

  data['RSI'] = rsi

  data['Buy'] = ((data['RSI'] < 30) & (data['RSI'].shift(1) >= 30))
  data['Sell'] = ((data['RSI'] > 70) & (data['RSI'].shift(1) <= 70))

  # Calculate profit from Feb 2022 to May 2022
  mask = (data.index >= start_date) & (data.index <= end_date)
  buy_mask = mask & data['Buy']
  sell_mask = mask & data['Sell']
  buy_price = data['Open'][buy_mask].iloc[0]
  sell_price = data['Open'][sell_mask].iloc[-1]
  shares = amount_invested / buy_price
  profit = shares * (sell_price - buy_price)
    
  RSI_Profit.append(round(profit,2))


# creating a dataframe out of the profits of 2 indicators

df_profit = pd.DataFrame({'Tickers': tickers, 'OBV_Profit': OBV_Profit, 'RSI_Profit': RSI_Profit})

print(df_profit)

# Observing which indicator is performing good

print("AVG OBV PROFIT",(df_profit['OBV_Profit'].sum())/5)

print("AVG RSI PROFIT",(df_profit['RSI_Profit'].sum())/5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
     Tickers  OBV_Profit  RSI_Profit
0       FSLR     4290.70     2507.44
1       ENPH     5539.49     1047.43
2        TAN     1470.05     -130.24
3        FAN      -55.39     -100.42
4  ORSTED.CO      652.70     1006.11
AVG OBV PROFIT 2379.5099999999998
AVG RSI PROFIT 866.064


### As OBV profit is greater than RSI profit **OBV is performing good**

In [ ]:
#@title Select the date range and indicator to know ROI { display-mode: "form" }
start_date = "2022-02-01" #@param {type:"date"}
end_date = "2022-05-31" #@param {type:"date"}
Indicator = "RSI" #@param ["RSI", "OBV"]
Ticker = "FAN" #@param ["FSLR", "TAN", "ENPH", "FAN", "ORSTED.CO"] {allow-input: true}
amount_invested = 10000 #@param {type:"number"}

# getting data

interval = "1d"

data = get_data(Ticker, start_date, end_date, interval)


if start_date < end_date:

  if Indicator == "OBV":

    # Calculate the On-Balance Volume (OBV) indicator
    close_prices = data['Adj Close']
    volume = data['Volume']
    change_in_price = close_prices.diff()

    # Add check for zero volume
    direction = pd.Series([0 if x == 0 else 1 if x >= 0 else -1 for x in change_in_price], index=change_in_price.index)
    obv = (direction * volume).cumsum()

    data['OBV'] = obv

    # Calculate the daily change in OBV
    data['OBV_change'] = data['OBV'].diff()

    # Define the buy and sell signals based on the OBV values
    data['signal'] = 0
    data.loc[data['OBV_change'] > 0, 'signal'] = 1  # buy signal
    data.loc[data['OBV_change'] < 0, 'signal'] = -1  # sell signal

    # Calculate the profit based on the buy/sell signals
    data['profit'] = data['Close'].pct_change() * data['signal'].shift(1)
    total_profit = data['profit'].sum()

    print(f'Total profit from ({start_date}) to ({end_date}):', total_profit * amount_invested)
    

  else:

    periods = 14

    close_delta = data['Close'].diff()
    gain = close_delta.where(close_delta > 0, 0)
    loss = -close_delta.where(close_delta < 0, 0)
    avg_gain = gain.rolling(window=periods).mean()
    avg_loss = loss.rolling(window=periods).mean()
    rs = avg_gain / avg_loss
    rsi = 100.0 - (100.0 / (1.0 + rs))

    data['RSI'] = rsi

    data['Buy'] = ((data['RSI'] < 30) & (data['RSI'].shift(1) >= 30))
    data['Sell'] = ((data['RSI'] > 70) & (data['RSI'].shift(1) <= 70))

    # Calculate profit from Feb 2022 to May 2022
    mask = (data.index >= start_date) & (data.index <= end_date)
    buy_mask = mask & data['Buy']
    sell_mask = mask & data['Sell']
    buy_price = data['Open'][buy_mask].iloc[0]
    sell_price = data['Open'][sell_mask].iloc[-1]
    shares = amount_invested / buy_price
    profit = shares * (sell_price - buy_price)
    print(f'Profit from {start_date} to {end_date}: ${profit:.2f}')

else:

  print("Start Date must be less than end date")

    
